<a href="https://colab.research.google.com/github/amit02-tech/FAKE-NEWS-AI-DETECTION/blob/main/trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from fastapi import FastAPI
import joblib
import pandas as pd

app = FastAPI()
model = joblib.load("model.joblib")
vectorizer = joblib.load("tfidf.joblib")

@app.post("/predict")
def predict(news: dict):
    text = news["title"] + " " + news["text"]
    X = vectorizer.transform([text])
    pred = model.predict(X)[0]
    return {"label": str(pred)}



In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Load data
df = pd.read_csv("/content/Fake_News_Detector.csv", on_bad_lines='skip', engine='python')
df.dropna(inplace=True)
X = df['title'] + " " + df['text']
y = df['label']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Logistic Regression model
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# Evaluate
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))

# Save artifacts
joblib.dump(model, "model.joblib")
joblib.dump(vectorizer, "tfidf.joblib")
print("Model and vectorizer saved!")

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      2693
           1       0.97      0.98      0.98      2323

    accuracy                           0.98      5016
   macro avg       0.98      0.98      0.98      5016
weighted avg       0.98      0.98      0.98      5016

Model and vectorizer saved!


In [10]:
from fastapi import FastAPI
import joblib

app = FastAPI()

# Load trained artifacts
model = joblib.load("model.joblib")
vectorizer = joblib.load("tfidf.joblib")

@app.post("/predict")
def predict(news: dict):
    text = news.get("title", "") + " " + news.get("text", "")
    X_vec = vectorizer.transform([text])
    pred = model.predict(X_vec)[0]
    return {"label": pred}
